In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.datasets import FashionMNIST
import pandas as pd
import time
from torch.utils.tensorboard import SummaryWriter

In [7]:
# Define the CNN Model
def create_cnn(kernel_size, pooling_type):
    class CNN(nn.Module):
        def __init__(self):
            super(CNN, self).__init__()
            self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_size, padding=kernel_size//2)
            self.pool1 = pooling_type(kernel_size=2, stride=2)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=kernel_size//2)
            self.pool2 = pooling_type(kernel_size=2, stride=2)
            self.fc1 = nn.Linear(64 * 7 * 7, 512)
            self.fc2 = nn.Linear(512, 10)

        def forward(self, x):
            x = self.pool1(torch.relu(self.conv1(x)))
            x = self.pool2(torch.relu(self.conv2(x)))
            x = x.view(-1, 64 * 7 * 7)
            x = torch.relu(self.fc1(x))
            x = self.fc2(x)
            return x

    return CNN()

In [8]:
# Define Training Function
def train_and_evaluate(model, optimizer, scheduler, train_loader, test_loader, criterion, device, epochs, config, results, patience=10):
    writer = SummaryWriter()
    model.to(device)
    start_time = time.time()
    best_accuracy = 0.0
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        scheduler.step(train_loss / len(train_loader))

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

    elapsed_time = time.time() - start_time
    results.append({
        'config': config,
        'epochs': epoch + 1,
        'best_accuracy': best_accuracy,
        'elapsed_time': elapsed_time
    })
    writer.close()

In [9]:
# Load FashionMNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=12, pin_memory=True, prefetch_factor=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=12, pin_memory=True, prefetch_factor=2)

# Results storage
results = []

In [10]:
# Experiment for Kernel Size
kernel_sizes = [3, 5, 7]
for kernel_size in kernel_sizes:
    config = f"KernelSize={kernel_size}"
    model = create_cnn(kernel_size, nn.MaxPool2d)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    scheduler = ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)
    train_and_evaluate(model, optimizer, scheduler, train_loader, test_loader, criterion, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), 50, config, results)

Epoch 1/50, Loss: 1.1317, Accuracy: 76.19%
Epoch 2/50, Loss: 0.5869, Accuracy: 80.45%
Epoch 3/50, Loss: 0.5145, Accuracy: 79.58%
Epoch 4/50, Loss: 0.4697, Accuracy: 82.08%
Epoch 5/50, Loss: 0.4376, Accuracy: 84.39%
Epoch 6/50, Loss: 0.4119, Accuracy: 82.93%
Epoch 7/50, Loss: 0.3904, Accuracy: 83.58%
Epoch 8/50, Loss: 0.3779, Accuracy: 86.22%
Epoch 9/50, Loss: 0.3635, Accuracy: 86.13%
Epoch 10/50, Loss: 0.3521, Accuracy: 86.01%
Epoch 11/50, Loss: 0.3417, Accuracy: 86.90%
Epoch 12/50, Loss: 0.3336, Accuracy: 87.42%
Epoch 13/50, Loss: 0.3243, Accuracy: 87.59%
Epoch 14/50, Loss: 0.3177, Accuracy: 87.94%
Epoch 15/50, Loss: 0.3097, Accuracy: 88.14%
Epoch 16/50, Loss: 0.3036, Accuracy: 88.05%
Epoch 17/50, Loss: 0.2969, Accuracy: 88.06%
Epoch 18/50, Loss: 0.2910, Accuracy: 88.45%
Epoch 19/50, Loss: 0.2849, Accuracy: 88.29%
Epoch 20/50, Loss: 0.2815, Accuracy: 88.52%
Epoch 21/50, Loss: 0.2767, Accuracy: 88.83%
Epoch 22/50, Loss: 0.2707, Accuracy: 89.39%
Epoch 23/50, Loss: 0.2673, Accuracy: 89.3

In [11]:
# Experiment for Pooling Types
pooling_types = [nn.MaxPool2d, nn.AvgPool2d]
for pooling_type in pooling_types:
    config = f"PoolingType={pooling_type.__name__}"
    model = create_cnn(3, pooling_type)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    scheduler = ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)
    train_and_evaluate(model, optimizer, scheduler, train_loader, test_loader, criterion, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), 50, config, results)

Epoch 1/50, Loss: 1.2045, Accuracy: 76.50%
Epoch 2/50, Loss: 0.5817, Accuracy: 79.49%
Epoch 3/50, Loss: 0.5049, Accuracy: 80.99%
Epoch 4/50, Loss: 0.4575, Accuracy: 82.71%
Epoch 5/50, Loss: 0.4245, Accuracy: 83.57%
Epoch 6/50, Loss: 0.4022, Accuracy: 85.09%
Epoch 7/50, Loss: 0.3822, Accuracy: 83.56%
Epoch 8/50, Loss: 0.3689, Accuracy: 86.74%
Epoch 9/50, Loss: 0.3546, Accuracy: 87.00%
Epoch 10/50, Loss: 0.3438, Accuracy: 86.84%
Epoch 11/50, Loss: 0.3340, Accuracy: 87.44%
Epoch 12/50, Loss: 0.3257, Accuracy: 87.85%
Epoch 13/50, Loss: 0.3175, Accuracy: 86.41%
Epoch 14/50, Loss: 0.3103, Accuracy: 87.03%
Epoch 15/50, Loss: 0.3032, Accuracy: 87.90%
Epoch 16/50, Loss: 0.2982, Accuracy: 88.15%
Epoch 17/50, Loss: 0.2912, Accuracy: 88.47%
Epoch 18/50, Loss: 0.2839, Accuracy: 88.53%
Epoch 19/50, Loss: 0.2818, Accuracy: 87.79%
Epoch 20/50, Loss: 0.2754, Accuracy: 89.15%
Epoch 21/50, Loss: 0.2714, Accuracy: 89.03%
Epoch 22/50, Loss: 0.2667, Accuracy: 89.27%
Epoch 23/50, Loss: 0.2607, Accuracy: 89.3

In [12]:
# Experiment for Epochs
epoch_list = [5, 50, 100, 250, 350]
for epochs in epoch_list:
    config = f"Epochs={epochs}"
    model = create_cnn(3, nn.MaxPool2d)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    scheduler = ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)
    train_and_evaluate(model, optimizer, scheduler, train_loader, test_loader, criterion, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), epochs, config, results)

Epoch 1/5, Loss: 1.1866, Accuracy: 72.72%
Epoch 2/5, Loss: 0.5958, Accuracy: 79.25%
Epoch 3/5, Loss: 0.5235, Accuracy: 80.08%
Epoch 4/5, Loss: 0.4759, Accuracy: 82.12%
Epoch 5/5, Loss: 0.4466, Accuracy: 83.80%
Epoch 1/50, Loss: 1.1368, Accuracy: 75.46%
Epoch 2/50, Loss: 0.5947, Accuracy: 76.44%
Epoch 3/50, Loss: 0.5278, Accuracy: 81.05%
Epoch 4/50, Loss: 0.4772, Accuracy: 82.30%
Epoch 5/50, Loss: 0.4411, Accuracy: 80.23%
Epoch 6/50, Loss: 0.4165, Accuracy: 82.56%
Epoch 7/50, Loss: 0.3971, Accuracy: 85.32%
Epoch 8/50, Loss: 0.3786, Accuracy: 85.98%
Epoch 9/50, Loss: 0.3657, Accuracy: 86.06%
Epoch 10/50, Loss: 0.3538, Accuracy: 86.49%
Epoch 11/50, Loss: 0.3434, Accuracy: 86.21%
Epoch 12/50, Loss: 0.3336, Accuracy: 85.47%
Epoch 13/50, Loss: 0.3247, Accuracy: 87.45%
Epoch 14/50, Loss: 0.3166, Accuracy: 86.95%
Epoch 15/50, Loss: 0.3100, Accuracy: 87.46%
Epoch 16/50, Loss: 0.3037, Accuracy: 88.34%
Epoch 17/50, Loss: 0.2978, Accuracy: 87.50%
Epoch 18/50, Loss: 0.2921, Accuracy: 88.71%
Epoch 1

In [13]:
# Experiment for Optimizers
optimizers = ['SGD', 'RMSprop', 'Adam']
for optimizer_name in optimizers:
    config = f"Optimizer={optimizer_name}"
    model = create_cnn(3, nn.MaxPool2d)
    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=0.01)
    scheduler = ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)
    train_and_evaluate(model, optimizer, scheduler, train_loader, test_loader, criterion, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), 50, config, results)

Epoch 1/50, Loss: 1.1326, Accuracy: 75.49%
Epoch 2/50, Loss: 0.5936, Accuracy: 77.43%
Epoch 3/50, Loss: 0.5222, Accuracy: 80.91%
Epoch 4/50, Loss: 0.4740, Accuracy: 83.21%
Epoch 5/50, Loss: 0.4413, Accuracy: 84.56%
Epoch 6/50, Loss: 0.4155, Accuracy: 83.31%
Epoch 7/50, Loss: 0.3963, Accuracy: 85.22%
Epoch 8/50, Loss: 0.3795, Accuracy: 85.82%
Epoch 9/50, Loss: 0.3653, Accuracy: 85.83%
Epoch 10/50, Loss: 0.3548, Accuracy: 84.84%
Epoch 11/50, Loss: 0.3441, Accuracy: 86.69%
Epoch 12/50, Loss: 0.3353, Accuracy: 87.18%
Epoch 13/50, Loss: 0.3262, Accuracy: 86.95%
Epoch 14/50, Loss: 0.3191, Accuracy: 87.02%
Epoch 15/50, Loss: 0.3106, Accuracy: 88.16%
Epoch 16/50, Loss: 0.3046, Accuracy: 87.76%
Epoch 17/50, Loss: 0.2990, Accuracy: 88.39%
Epoch 18/50, Loss: 0.2938, Accuracy: 88.36%
Epoch 19/50, Loss: 0.2873, Accuracy: 88.31%
Epoch 20/50, Loss: 0.2820, Accuracy: 87.63%
Epoch 21/50, Loss: 0.2762, Accuracy: 88.94%
Epoch 22/50, Loss: 0.2720, Accuracy: 88.83%
Epoch 23/50, Loss: 0.2668, Accuracy: 89.4

In [14]:
# Display results
results_df = pd.DataFrame(results)
print(results_df)

                   config  epochs  best_accuracy  elapsed_time
0            KernelSize=3      50          90.89    154.327961
1            KernelSize=5      50          91.06    155.015474
2            KernelSize=7      50          91.06    156.605840
3   PoolingType=MaxPool2d      50          91.28    159.756756
4   PoolingType=AvgPool2d      50          88.14    159.158159
5                Epochs=5       5          83.80     16.051471
6               Epochs=50      50          90.86    165.196237
7              Epochs=100      67          91.16    222.446367
8              Epochs=250      92          92.06    311.812903
9              Epochs=350      52          90.91    178.780155
10          Optimizer=SGD      50          90.99    173.347311
11      Optimizer=RMSprop      25          88.97     89.024834
12         Optimizer=Adam      33          89.47    119.064227
